Since we have enough rare class data(at least 1K).Let handle imbalanced data with down sample before training.


# load package


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

from siuba.siu import call
from siuba import _, mutate, filter, group_by, summarize,show_query

import time
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

# data

## download data

https://www.kaggle.com/datasets/vetrirah/customer


In [ ]:
import pandas as pd

df_tran_raw=pd.read_csv('data/Train.csv')

df_test_raw=pd.read_csv('data/Test.csv')

## data EDA



## Data Wrangling


In [ ]:
df_tran_raw.head()

In [ ]:
df_train=df_tran_raw.drop('ID', axis=1)

In [ ]:
df_test=df_test_raw.drop('ID', axis=1)

In [ ]:
df_train.head()

In [ ]:
df_train >> group_by(_.Segmentation)  >> summarize(n = _.shape[0])

## split data

80% training / 10% validation/ 10% testing

![](images/1_Nv2NNALuokZEcV6hYEHdGA.webp){width="520"}


In [ ]:
Y=df_train['Segmentation']
X=df_train.drop('Segmentation', axis=1)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
import collections, numpy
collections.Counter(Y)

inverse_transform for testing only


In [ ]:
inv = le.inverse_transform(Y)

In [ ]:
import collections, numpy
collections.Counter(inv)

In [ ]:
training_size=0.8
validation_size=0.1
testing_size=0.1



X_train, X_val, Y_train, Y_val= train_test_split(X, Y, test_size=validation_size, random_state=1)

X_train, X_test, Y_train, Y_test= train_test_split(X_train, Y_train, test_size=testing_size/training_size, random_state=1) 

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
len(X_train)/(len(X_train) +len(X_val) +len(X_test) )

In [ ]:
len(X_val)/(len(X_train) +len(X_val) +len(X_test) )

In [ ]:
len(X_test)/(len(X_train) +len(X_val) +len(X_test) )

## categorical_cols and numerical_cols


In [ ]:
categorical_cols = [cname for cname in X_train 
                    if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]
                    
                    
numerical_cols = numerical_cols = [cname for cname in X_train 
                    if X_train[cname].dtype in ['int64', 'float64']]

In [ ]:
print("The total number of categorical columns:", len(categorical_cols))
print("The total number of numerical columns:", len(numerical_cols))

In [ ]:
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_val = X_val[my_cols].copy()
X_test= X_test[my_cols].copy()

my_cols
#X_final = df_test[my_cols].copy()

## Pipelines for Data Preprocessing


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

### numerical_transformer


In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler())
])

### categorical_transformer


In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ('imputer_cat', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)])

# model


## define model


### XGB model


In [ ]:
import xgboost
print(xgboost.__version__)

Since its multi calss classification so add `objective='multi:softmax'`

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(objective='multi:softprob',num_class=4)
xgb_model

### Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier()
random_forest_model

### Logistic Regression model


In [ ]:
from sklearn.linear_model import LogisticRegression
LogisticRegression_model = LogisticRegression(solver='liblinear')
LogisticRegression_model

## define pipline


In [ ]:
pipeline_xgb = Pipeline(
  steps=[
         ('preprocessor', preprocessor), 
         ('model', xgb_model)
         ]
)

pipeline_rf = Pipeline(
  steps=[
         ('preprocessor', preprocessor), 
         ('model', random_forest_model)
         ]
)

pipeline_lr = Pipeline(
  steps=[
         ('preprocessor', preprocessor), 
         ('model', LogisticRegression_model)
         ]
)

## define GridSearch


In [ ]:
parameters_xgb= {
        'model__learning_rate': [0.01, 0.02,0.08,0.1],
        'model__max_depth': [3, 5, 7,8,9,10],
        'model__min_child_weight': [1, 3,5,8],
        'model__subsample': [0.5, 0.7,0.9],
        
       # 'model__colsample__bytree': [0.5, 0.7],
       
        'model__n_estimators' : [100, 200],
        'model__objective': ['reg:squarederror']
    }


Grid_xgb = HalvingGridSearchCV(pipeline_xgb
                ,parameters_xgb 
                #,scoring='merror'
                ,max_resources=100
                , cv=10, n_jobs=-1)
                
                
parameters_rf = {'model__max_depth':[20,30,40],
                 'model__n_estimators':[200,250],
                 'model__min_samples_leaf':[1,2,3]
                 }                
                

Grid_rf = HalvingGridSearchCV(pipeline_rf
                ,parameters_rf
                #,scoring='merror'
                ,max_resources=100
                , cv=10, n_jobs=-1)


## train model


In [ ]:
start_time = time.time()


Grids = [Grid_xgb, Grid_rf,pipeline_xgb,pipeline_rf,pipeline_lr]



for Grid in Grids:
    Grid.fit(X_train,Y_train)


end_time = time.time()
duration = end_time - start_time
duration

print("trainning time:",duration)

## Preformance

### dummy Preformance

#### dummy 1 : there are 3 class, so each class random 33.33%

In [ ]:
import random
from array import array

mylist = []

for i in range(0,len(Y_test)):
    x = random.randrange(0, 4)
    mylist.append(x)
    
Y_dummy1=numpy.array(mylist)

In [ ]:
import collections, numpy
c=collections.Counter(Y_dummy1)
df = pd.DataFrame.from_records(list(dict(c).items()), columns=['class','count']).sort_values('class')
df

In [ ]:
print(classification_report(le.inverse_transform(Y_test), le.inverse_transform(Y_dummy1)))

#### dummy 2 : there are 3 class, random assign to each share:

each class in Y_test 


In [ ]:
import collections, numpy
c=collections.Counter(Y_test)
df = pd.DataFrame.from_records(list(dict(c).items()), columns=['class','count']).sort_values('class')
df

In [ ]:
import random
Y_dummy2=numpy.array(random.choices([0, 1, 2,3], [218/sum(Y_test), 208/sum(Y_test), 218/sum(Y_test),264/sum(Y_test)], k=len(Y_test)))

In [ ]:
import collections, numpy
c=collections.Counter(Y_dummy2)
df = pd.DataFrame.from_records(list(dict(c).items()), columns=['class','count']).sort_values('class')
df

In [ ]:
print(classification_report(le.inverse_transform(Y_test), le.inverse_transform(Y_dummy2)))

### model Preformance

In [ ]:
grid_dict = {0: 'XGB', 1: 'random forest', 2: 'XGB non tune',3: 'ramdon forest non tune',4:'Logistic regression non tune' }

for i, model in enumerate(Grids):
    print('{} Test Accuracy: {}'.format(grid_dict[i],
    model.score(X_test,Y_test)))
    #print('{} Best Params: {}'.format(grid_dict[i], model.best_params_))

In [ ]:
best_ml=Grid_xgb.best_estimator_

In [ ]:
type(best_ml)

### model feature importances

In [ ]:
vi=best_ml.steps[1][1].feature_importances_
vi

In [ ]:
## get pipeline feature names
var_name=best_ml[:-1].get_feature_names_out()

In [ ]:
dataset = pd.DataFrame({'feature': var_name, 'importances': vi}, columns=['feature', 'importances']).sort_values('importances',ascending=False)
dataset

In [ ]:
#Using predict method to test the model
Y_pred_dt = best_ml.predict(X_test) #always gets x and retuns y
#Y_pred_dt

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred_dt))

reverse target label back to A B C D

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(le.inverse_transform(Y_test), le.inverse_transform(Y_pred_dt)))

## save model


In [ ]:
from joblib import dump, load
dump(Grid_xgb, 'trained_grid_multi_calss.joblib', compress=True)  

## load model


In [ ]:
model_reload = load('trained_grid_multi_calss.joblib') 

In [ ]:
best_ml=model_reload.best_estimator_

## final prediction


In [ ]:
Y_pred_dt_final =best_ml.predict(X_val) #always gets x and retuns y


Y_pred_dt_final_lable=le.inverse_transform(Y_pred_dt_final)
Y_pred_dt_final_lable[0:5]

In [ ]:
import collections, numpy
c=collections.Counter(Y_pred_dt_final_lable)
df = pd.DataFrame.from_records(list(dict(c).items()), columns=['class','count']).sort_values('class')
df

# reference:

https://www.kaggle.com/code/mittalvasu95/multi-class-classification-c101
